In [2]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
import os

from coeditor.dataset import TokenizedEditDataset
from coeditor.model import *
from coeditor.encoding import TokenizedEdit, decode_tokens, WindowArgs, tokens_to_change
import shutil
import random
from prepare_data import make_or_load_datasets

os.chdir(proj_root())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
wargs = WindowArgs(4096)

# test_data_name = "medium"
test_data_name = "SPOT"
datasets = make_or_load_datasets(test_data_name)
test_data = datasets["test"]
test_data = test_data.map(lambda e: e.truncate_ctx(wargs))

rep_edits = pfilter(TokenizedEdit.is_repetitive_edit, test_data.all_edits())
small_edits = pfilter(TokenizedEdit.is_small_edit, test_data.all_edits())
print("Total edits: ", len(list(test_data.all_edits())))
print("Repetitive edits: ", len(rep_edits))
print("Small edits: ", len(small_edits))


Starting task: Load dataset from: /mnt/nas/jiayi/coeditor/datasets/SPOT/AnalysisBasedEditEncoder(window=(max_window_size=4096), extra_ctx_size=1000, extra_ctx_names=('usees', 'post-usees'))
(0.2s) Finished task: Load dataset from: /mnt/nas/jiayi/coeditor/datasets/SPOT/AnalysisBasedEditEncoder(window=(max_window_size=4096), extra_ctx_size=1000, extra_ctx_names=('usees', 'post-usees'))


filtering: 100%|██████████| 1343/1343 [00:00<00:00, 8178.41it/s]


Total edits:  1343
Repetitive edits:  87
Small edits:  978


In [4]:
# model_dir = get_model_dir(trained=True) / "coeditor-medium-collapse-shuffle"
model_dir = get_model_dir(trained=True) / "coeditor-medium-analysis-post_usees"
model = CoeditorModel.load_pretrained(model_dir)
model.to("cuda:0")

eval_args = EvalArgs(4096 * 2, WindowArgs(4096))


In [7]:
rep_data = TokenizedEditDataset.from_edits(rep_edits)
dec_args = DecodingArgs(num_beams=1)
rep_result = model.predict_on_data(rep_data, eval_args, dec_args)
display(rep_result.exact_match_accuracy())

decoding: 100%|██████████| 36/36 [00:59<00:00,  1.66s/batch]


(mean=0.77011, weight=87)

In [5]:
small_data = TokenizedEditDataset.from_edits(small_edits)
dec_args = DecodingArgs(num_beams=1)
dec_result = model.predict_on_data(small_data, eval_args, dec_args)
display(dec_result.exact_match_accuracy())


decoding: 100%|██████████| 365/365 [09:11<00:00,  1.51s/batch]


(mean=0.14213, weight=978)

In [8]:
max_saved_samples = 200
random.seed(42)
exs_to_save = list(range(len(dec_result.predictions)))
random.shuffle(exs_to_save)
out_dir = Path("output/inspect_coeditor") / model_dir.name
dec_result.save_examples_to_dir(out_dir, exs_to_save[:max_saved_samples])

saving examples: 100%|██████████| 50/50 [00:00<00:00, 81.92it/s]
